In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import requests
import time

In [ ]:

# Leer el archivo de Spotify (ajusta el path si es necesario)
df_artists = pd.read_csv("artists.csv")

artistas_raw = df_artists['artist'].dropna().unique()
artistas = set()

# Función para separar artistas con múltiples delimitadores
def split_artists(texto):
    # Reemplazar "Featuring" por un delimitador temporal (e.g., ";") y limpiar
    texto = re.sub(r'\s*Featuring\s*', ',', texto, flags=re.IGNORECASE)
    # Separar por ";" y "&"
    partes = re.split(r'[,&]', texto)
    # Limpiar cada artista y devolver lista
    return [artista.strip() for artista in partes if artista.strip()]

# Procesar cada grupo de artistas
for grupo in artistas_raw:
    for artista in split_artists(grupo):
        artistas.add(artista)

print(f"🎵 Total de artistas únicos: {len(artistas)}")

🎵 Total de artistas únicos: 2967


In [ ]:

# Consultar MusicBrainz para cada artista
headers = {'User-Agent': 'ETLWorkshop/1.0 (tucorreo@dominio.com)'}
datos_artistas = []

for artist in list(artistas):  # puedes ajustar el número para pruebas
    query = artist.replace(" ", "+")
    url = f"https://musicbrainz.org/ws/2/artist/?query=artist:{query}&fmt=json&limit=1"
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data['artists']:
            a = data['artists'][0]
            datos_artistas.append({
                'artist_name': artist,
                'artist_id': a.get('id'),
                'country': a.get('country'),
                'begin_date': a.get('life-span', {}).get('begin'),
                'end_date': a.get('life-span', {}).get('end'),
                'type': a.get('type'),
                'gender': a.get('gender'),
                'disambiguation': a.get('disambiguation'),
            })
    else:
        print(f"❌ Error con {artist}: {response.status_code}")
    
    time.sleep(1)

# Guardar como CSV
df_api = pd.DataFrame(datos_artistas)
df_api.to_csv("musicbrainz_artists.csv", index=False)

print("✅ Datos de MusicBrainz guardados en api_data.csv.")


✅ Datos de MusicBrainz guardados en api_data.csv.


In [32]:
df = pd.read_csv("musicbrainz_artists.csv")
df.head()

,artist_name,artist_id,country,begin_date,end_date,type,gender,disambiguation
0,Rhonda Vincent And The Rage,b6310fcc-30ea-41a9-9cda-1931f569d76f,US,NaN,NaN,Group,NaN,NaN
1,Yolanda Kondonassis,73e9c3b8-ef39-48e5-8b08-f746dcf0c765,US,1963,NaN,Person,female,harpist
2,John Hartford,3178cc4c-cd9f-4842-96fb-28bf9cfec591,US,1937-12-30,2001-06-04,Person,male,NaN
3,Sheryl Crow,80ccfede-c258-4575-a7ad-c982e9932e0f,US,1962-02-11,NaN,Person,female,American singer-songwriter and musician
4,Neal Hefti,75770f14-cbe0-4b06-8825-14797098ff2a,US,1922-10-29,2008-10-11,Person,male,NaN
